In [1]:
"""
Import des modules
"""
import numpy as np


In [14]:
"""
    Generation des donnees
"""
eta_edp = 2
def generate_data(n,m):
    train_data   = []
    train_target = []
    test_data    = []
    test_target  = []

    #Generate train data
    for i in range(1,n+1):
        j = np.random.rand()
        train_data.append([j])
        f = eta_edp*np.exp(eta_edp*j)
        train_target.append([f])
    #Generate Test data
    for i in range(1,m+1):
        j = np.random.rand()
        test_data.append([j])
        f = eta_edp*np.exp(eta_edp*j)
        test_target.append([f])

    return train_data,train_target,test_data,test_target

"""
Test
"""
td,tt,td1,tt1 = generate_data(5,3)
print('Train_data = ',td)
print('Train_target = ',tt)
print('Test_data = ',td1)
print('Test_target = ',tt1)

Train_data =  [[0.4487857452076789], [0.8190119784963734], [0.9281564925175207], [0.7854812431426049], [0.34822596145768325]]
Train_target =  [[4.9072743770186715], [10.289985467512555], [12.8001919332075], [9.622553507689984], [4.0132408361470855]]
Test_data =  [[0.5193949864789646], [0.20820346884660923], [0.48978672208684493]]
Test_target =  [[5.65159131128408], [3.0330057310006153], [5.326639889919498]]


In [21]:
def sigmoid(z):
    """
        Activation/Sigmoid
    """
    return 1 / (1 + np.exp(-z))


def d_sigmoid(z):
    """
        Derivative of the activation/Sigmoid

    """
    return sigmoid(z) * (1 - sigmoid(z))


def ANN(x, v,w,b,d):
    """
        Compute the output of the ANN
        **input: **
            *x : features
            *v : first layer weights
            *w : output node weight
            *b : bias of first layer
            *d : output node bias
    """
    y = sigmoid(np.dot(v,np.transpose(x))+b)
    print(np.shape(y))
    return np.matmul(np.transpose(w), y) + d

def dx_ANN(x, v,w,b,d):
    """
        Compute the output of the derivated ANN
        **input: **
            *x : features
            *v : first layer weights
            *w : output node weight
            *b : bias of first layer
            *d : output node bias
    """
    y = np.dot(v,np.transpose(x))+b
    z = d_sigmoid(np.matmul(np.transpose(w), y) + d)
    return np.dot(np.transpose(v),w)*z


def jac_ANN(x,v,w,b,d):
    """
        Compute the output of the Jacobian(weights) of the ANN
        **input: **
            *x : features
            *v : first layer weights
            *w : output node weight
            *b : bias of first layer
            *d : output node bias
    """
    t = np.shape(x)[0]
    s = np.shape(v)[0]
    jac = np.zeros([t,3*s+1])
    print(np.shape(jac))
    for j in range(t):
        jac[j][0:s] =  np.matmul(np.transpose(w),d_sigmoid(np.matmul(v,x[j])+b))
        jac[j][s:2*s] = x[j]*jac[j][0:s]
        jac[j][2*s:3*s] = np.reshape(sigmoid(x[j]*v+b),[s])
        jac[j][3*s] = 1
    return jac



In [16]:
"""
Levenberg-Marquardt method for large scale nonlinear least squares problems 
    Input:
        x     : variable with respect to which we solve for (weights)
        y     : training sample
        F     : function (-nabla ANN(u(y,x))-f(x))
        J     : function handle that computes Jx and J'x
        tol   : stopping tolerance 
        maxit : max number of iteration
        """

def LM(x,y,F,J,tol,maxit,LAMBDA) :
    """
    Initialisation
    """

    # maximum, minimum  of LAMBDA
    LAMBDA_max = 1.e+6
    LAMBDA_min = 1.e-4
    LAMBDAk    = LAMBDA

    #Iterations counter
    nbiter = 0

    #Parameters
    eta    = 0.1
    gamma1 = 0.85
    gamma2 = 1.5

    #Iterate
    xk = x

    #Function, Jacobian and gradient evaluation
    fx   = F(xk,y)
    jac  = J(xk,y)
    grad =  np.matmul(np.transpose(jac),fx)

    #Actual norm of gradient
    normgrad = np.linalg.norm(grad)

    """ 
    Computation
    """
    while (normgrad > tol) & (nbiter < maxit):
        #inc iter
        nbiter += 1

        #Compute the step pk
        n         = np.shape(jac)[1]
        jacjac    = np.matmul(np.transpose(jac),jac)
        jac_reg   = np.add(jacjac,np.dot(np.eye(n),LAMBDAk))
        pk        = np.linalg.solve(jac_reg,np.dot(grad,-1))

        #Compute rhok
        xkaux = np.add(xk,pk)
        ared  = np.linalg.norm(fx)**2 - np.linalg.norm(F(xkaux,y))**2
        pred  = np.linalg.norm(fx)**2
        pred  = pred - np.linalg.norm(np.add(fx,np.matmul(jac,pk)))**2
        rhok  = ared/pred

        #update iterate
        if (rhok > eta):
            xk      = xkaux
            fx      = F(xk,y)
            jac     = J(xk,y)
            grad    = np.matmul(np.transpose(jac),fx)
            LAMBDAk = gamma1*LAMBDAk
            normgrad = np.linalg.norm(grad)
        else:
            LAMBDAk = gamma2*LAMBDAk
    return xk,nbiter


"""
Tests
"""

A = np.array([[2,0],[0,4]])
x = np.array([0.2,0.2])
y = np.array([1,1])

def Ftest(x,y):
    Ax = np.matmul(A,x)
    return np.add(Ax,np.dot(y,-1))

def Jtest(x,y):
    return A
x,iters = LM(x,y,Ftest,Jtest,1e-6,1000,0.05)
print('nbiter = ',iters)
print('sol    = ',x)


nbiter =  4
sol    =  [0.5  0.25]


In [11]:
def init_variables(n):
    """
        Init model variables (weights and bias)
    """
    v = np.random.randn(n,1)
    w = np.random.randn(n,1)
    b = np.ones([n,1])
    d = 0


    return w,v,b,d

def cost(predictions, targets):
    """
        Compute the cost of the model
        **input: **
            *predictions: (Numpy vector) y
            *targets: (Numpy vector) t
    """
    return np.mean((predictions - targets) ** 2)

In [17]:
#Variable global
n  = 10000 #Nombres de paquets de données générées
p  = 1000 # Nombres de paquets de données de test générées 
m  = 200 #Nombres de neurones dans le hidden layer
#Recuperation des donnees
train_data,train_target, test_data, test_target = generate_data(n,2)

In [37]:
def merge_weights(v,w,b,d):
    return np.concatenate((v,w,b,d))

def break_weights(weights,size):
    v = weights[0:size]
    w = weights[size:2*size]
    b = weights[2*size:3*size]
    d = np.array([weights[3*size]])
    return v,w,b,d
    
"""
Test
"""
print(merge_weights(np.array([1,2,3]),np.array([2,5]),np.array([1]),np.array([4])))
print(break_weights(np.array([1,2,3,4,5,6,7,8,9,10]),3))

[1 2 3 2 5 1 4]
(array([1, 2, 3]), array([4, 5, 6]), array([7, 8, 9]), array([10]))


In [38]:
"""
Method used to train the model using Levenberg Marquadt Method
   **input: **
           *features
            *targets
            *v : first layer weights
            *w : output node weight
            *b : bias of first layer
            *d : output node bias
    **return dw,dv,db,dd*
            *update first layer weights
            *update output node weight
            *update first layer bias
            *update output node weight
"""
def train(features, targets, w,v,b,d):
    epochs = 100
    learning_rate = 0.1
    
    # Print current Accuracy
    predictions = predict(features, weights, bias)
    print("Accuracy = %s" % np.mean(predictions == targets))


    for epoch in range(epochs):
        # Compute and display the cost every 10 epoch
        if epoch % 10 == 0:
            predictions = activation(pre_activation(features, weights, bias))
            print("Current cost = %s" % cost(predictions, targets))

        # Appel Levenberg Marquadt
        weights = merge_weights(w,v,b,d)
        weights = LM(weights,features,F,J,1e-6,10000,0.05)

    # Print current Accuracy
    predictions = predict(features, weights, bias)
    print("Accuracy = %s" % np.mean(predictions == targets))